In [19]:
#importing libraries
import pandas as pd 
import csv, sqlite3

**Creating a connection to a database**

In [33]:
con = sqlite3.connect('../input/sql-murder-mystery-database/sql-murder-mystery.db')

**Overview of columns in crime_scene_report**

In [35]:
q1 = 'select * from crime_scene_report limit 1;'
pd.read_sql_query(q1, con)

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC


**Using available information to restrict result set**

In [39]:
q2 = "select description from crime_scene_report where date='20180115' and type = 'murder' and city='SQL City';"
pd.read_sql_query(q2,con)

,description
0,Security footage shows that there were 2 witne...


**Retrieving testimonies from both witnesses**

In [42]:
#first witness testimony
q3 = "select i.transcript from interview i inner join person p on i.person_id = p.id where p.address_street_name\
    like 'Northwestern Dr%' order by address_number desc limit 1"
pd.read_sql_query(q3, con)

,transcript
0,I heard a gunshot and then saw a man run out. ...


testimony 1:
I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".

In [48]:
#second witness testimony
q4 = "select i.transcript, p.name from interview i inner join person p on i.person_id = p.id where p.name like 'Annabel%' \
    and p.address_street_name like 'Franklin Ave%'"
pd.read_sql_query(q4, con)

,transcript,name
0,"I saw the murder happen, and I recognized the ...",Annabel Miller


testimony 2:
I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.

**Looking further into testimony 1**

*the membership number on the bag started with "48Z"*

*Only gold members have those bags.*

*The man got into a car with a plate that included "H42W"*

In [88]:
q5 = """select gfc.check_in_time, gfc.check_out_time,gfm.name, gfm.person_id
        from get_fit_now_check_in gfc 
        inner join get_fit_now_member gfm 
        on gfm.id = gfc.membership_id
        where gfc.membership_id like '48Z%' and gfm.membership_status='gold' """
pd.read_sql_query(q5,con)  

,check_in_time,check_out_time,name,person_id
0,1600,1730,Joe Germuska,28819
1,1530,1700,Jeremy Bowers,67318


In [81]:
q6 = """select distinct dl.id as drivers_license_id from drivers_license dl
        where dl.plate_number like '%H42W%' """
pd.read_sql_query(q6,con)  

,drivers_license_id
0,183779
1,423327
2,664760


**Looking further into testimony 2**

In [83]:
q7 = """select distinct gfc.membership_id 
        from get_fit_now_check_in gfc 
        where gfc.check_in_date = '20180109'   """
pd.read_sql_query(q7,con)  

,membership_id
0,X0643
1,UK1F2
2,XTE42
3,1AE2H
4,6LSTG
5,7MWHJ
6,GE5Q8
7,48Z7A
8,48Z55
9,90081


**Drawing conclusions**

In [91]:
q8 = "select p.name as Killer_Name, p.id as Killer_ID from person p where p.license_id in (183779,423327,664760) and p.id in (28819,67318) "
pd.read_sql_query(q8,con)  

,Killer_Name,Killer_ID
0,Jeremy Bowers,67318


**The killer is revealed!! Jeremy Bowers**

**Retrieving killer's transcipt for further details**

In [92]:
q9 = "select * from interview where person_id =67318"
pd.read_sql_query(q9,con)

,person_id,transcript
0,67318,I was hired by a woman with a lot of money. I ...


Killers Transcipt: 
I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017

**Solving the mystery further**

**Retrieving drivers license ids**

In [101]:
q10 = "select id from drivers_license where gender='female' and hair_color='red' and car_make ='Tesla' and car_model='Model S' "
pd.read_sql_query(q10,con)

,id
0,202298
1,291182
2,918773


**Retrieving the associated person ids**

In [103]:
q11 = """select p.id from person p 
        where p.license_id in (202298,291182,918773)"""
pd.read_sql_query(q11,con)

,id
0,78881
1,90700
2,99716


**Filtering based on person ids and event location**

In [104]:
q12 = """select * from 
    facebook_event_checkin
    where event_name ='SQL Symphony Concert' and person_id in (78881,90700,99716) """
pd.read_sql_query(q12,con)
#this confirms the woman attended three times

,person_id,event_id,event_name,date
0,99716,1143,SQL Symphony Concert,20171206
1,99716,1143,SQL Symphony Concert,20171212
2,99716,1143,SQL Symphony Concert,20171229


**Retrieving woman's name and info**

In [106]:
q13 = """select * from person where id = 99716 """
pd.read_sql_query(q13,con)
#this confirms the woman attended three times

,id,name,license_id,address_number,address_street_name,ssn
0,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


***The mastermind behind the murder is Miranda Priestly***!!